<a href="https://colab.research.google.com/github/dwahast/Deep-learning/blob/master/MLP_PyTorch_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivos deste trabalho
- Familiarizar-se com a biblioteca PyTorch
- Definir arquiteturas MLP simples em PyTorch
- Treinar utilizando CIFAR10, testando diferentes arquiteturas, parâmetros, funções de loss e otimizadores
- Comparar os resultados obtidos utilizando apenas Perpceptrons

In [0]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [14]:
# Carregar os datasets

transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
train_loader = DataLoader(dataset=dataset_train, shuffle=True)
test_loader = DataLoader(dataset=dataset_test, shuffle=False)

In [0]:
# Definir a arquitetura MLP

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(32*32, 20)
        self.fc2 = nn.Linear(20, 10)
        self.activation_function = nn.Sigmoid()
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        return x

In [5]:
model = MLP()
print(model)

MLP(
  (fc1): Linear(in_features=1024, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (activation_function): Sigmoid()
)


In [0]:
# Definir otimizador e loss
# Nota: testar outros otimizadores e funções de loss (em particular cross entropy)

optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
loss_fn = torch.nn.MSELoss()
#loss_fn = torch.nn.CrossEntropyLoss()

In [0]:
def one_hot(label, output_size):
    
    label_select = np.zeros(output_size)
    label_select[label]=1
    
    return torch.Tensor(label_select)

In [0]:
# Realizar o treinamento aqui
def fit(epochs):
  losses = []
  for epoch in range(epochs):
    model.train() # Set model to TRAIN "mode" (can be set to False for Test)
    batch_losses = []
    for image,label in train_loader:
      optimizer.zero_grad()  # cleaning gradients between mini batches
      weights = model(image) 
      label_target = one_hot(label,10) # one_hot to set the training class in the escalar
      loss = loss_fn(weights,label_target)
      loss.backward() # Backpropagation 
      optimizer.step() # Optimization Method
      batch_losses.append(loss.item())

    losses.append(np.mean(batch_losses)) # Append mean losses for each epoch
    #print(epoch, np.mean(batch_losses)) # Print Loss for each epoch

  plt.plot(losses)
  pass

In [0]:
# Avaliar o modelo aqui (no conjunto de teste)
fit(100)